**Train**

In [1]:
!pip install pycocotools

In [2]:
import os
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count: int  = 0
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
# BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017'  
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
ANNOTATION_FILE = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained CLIP Processor and CLIP model
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    # Move inputs to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        features = outputs.squeeze().cpu().numpy()
        
    # Ensure the features are in the expected shape (e.g., 512 for CLIP ViT-B/32)
    if features.ndim == 1:
        features = features.reshape(1, -1)
    
    return features

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

for i, img_id in enumerate(image_ids):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    # Ensure features are not empty
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
#         print(f"No labels for image: {image_path}")
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if i % 11800 == 0:
        print(f"{i} images processed")

# Flatten the features_list if necessary
train_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
train_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", train_features_array.shape)
print("Labels array shape:", train_labels_array.shape)

np.save(os.path.join(OUTPUT_PATH, "clip_train_features.npy"), train_features_array)
np.save(os.path.join(OUTPUT_PATH, "clip_train_labels.npy"), train_labels_array)
# Print images without labels
print("Total Images without labels: ", no_label_count)
print("Image without labels: ", images_without_labels)


Using device: cuda
loading annotations into memory...
Done (t=21.97s)
creating index...
index created!


2024-05-27 16:00:33.388873: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 16:00:33.388935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 16:00:33.390546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

0 images processed
11800 images processed
23600 images processed
35400 images processed
47200 images processed
59000 images processed
70800 images processed
82600 images processed
94400 images processed
106200 images processed
118000 images processed
Features array shape: (117266, 512)
Labels array shape: (117266, 80)
Total Images without labels:  1021
Image without labels:  ['/kaggle/input/coco-2017-dataset/coco2017/train2017/000000262284.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000108169.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000176649.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000570045.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000191501.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000125997.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000069911.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000375363.jpg', '/kaggle/input/coco-2017-dataset/coco2017/train2017/0

**Test**

In [ ]:
import os
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count: int = 0

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
# BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017'  
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/test2017'
ANNOTATION_FILE = '/kaggle/input/annotation-2017/image_info_test2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained CLIP Processor and CLIP model
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    # Move inputs to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        features = outputs.squeeze().cpu().numpy()
        
    # Ensure the features are in the expected shape (e.g., 512 for CLIP ViT-B/32)
    if features.ndim == 1:
        features = features.reshape(1, -1)
    
    return features

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

for i, img_id in enumerate(image_ids):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    # Ensure features are not empty
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
#         print(f"No labels for image: {image_path}")
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if i % 4000 == 0:
        print(f"{i} images processed")

# Flatten the features_list if necessary
test_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
train_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", test_features_array.shape)
print("Labels array shape:", test_labels_array.shape)

np.save(os.path.join(OUTPUT_PATH, "clip_test_features.npy"), test_features_array)
np.save(os.path.join(OUTPUT_PATH, "clip_test_labels.npy"), test_labels_array)

# Print images without labels
print("Total Images without labels:", no_label_count)
print("Images without labels: ",images_without_labels)


**Val**

In [4]:
import os
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from pycocotools.coco import COCO

no_label_count: int  = 0
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# File paths and directories (Kaggle specific)
# BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017'  
IMAGE_DIR = '/kaggle/input/coco-2017-dataset/coco2017/val2017'
ANNOTATION_FILE = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
OUTPUT_PATH = '/kaggle/working'

# Load COCO annotations
coco = COCO(ANNOTATION_FILE)
image_ids = coco.getImgIds()

# Load the pretrained CLIP Processor and CLIP model
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Move model to GPU if available
model.to(device)

def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    # Move inputs to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        features = outputs.squeeze().cpu().numpy()
        
    # Ensure the features are in the expected shape (e.g., 512 for CLIP ViT-B/32)
    if features.ndim == 1:
        features = features.reshape(1, -1)
    
    return features

# Feature extraction loop
features_list = []
labels_list = []
images_without_labels = []

for i, img_id in enumerate(image_ids):
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])
    features = extract_features(image_path)
    
    # Ensure features are not empty
    if features.size == 0:
        print(f"Empty features for image: {image_path}")
        continue
    
    # Get annotations for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    label = [ann['category_id'] for ann in anns]  
    
    if not label:
#         print(f"No labels for image: {image_path}")
        no_label_count += 1
        images_without_labels.append(image_path)
        continue

    features_list.append(features)
    labels_list.append(label)
    
    if i % 500 == 0:
        print(f"{i} images processed")

# Flatten the features_list if necessary
val_features_array = np.vstack(features_list)

# Convert labels_list to one-hot encoding
unique_labels = sorted(set(label for labels in labels_list for label in labels))
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def convert_to_one_hot(labels, label_map):
    one_hot_labels = np.zeros(len(label_map))
    for label in labels:
        if label in label_map:  # Ensure label is in the label map
            one_hot_labels[label_map[label]] = 1
    return one_hot_labels

one_hot_labels_list = [convert_to_one_hot(labels, label_map) for labels in labels_list]
val_labels_array = np.array(one_hot_labels_list)

# Ensure train_features_array and train_labels_array are correctly shaped
print("Features array shape:", val_features_array.shape)
print("Labels array shape:", val_labels_array.shape)

np.save(os.path.join(OUTPUT_PATH, "clip_val_features.npy"), val_features_array)
np.save(os.path.join(OUTPUT_PATH, "clip_val_labels.npy"), val_labels_array)
# Print images without labels
print("Total Images without labels: ", no_label_count)
print("Image without labels: ", images_without_labels)


Using device: cuda
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
0 images processed
500 images processed
1000 images processed
1500 images processed
2000 images processed
2500 images processed
3000 images processed
3500 images processed
4000 images processed
4500 images processed
Features array shape: (4952, 512)
Labels array shape: (4952, 80)
Total Images without labels:  48
Image without labels:  ['/kaggle/input/coco-2017-dataset/coco2017/val2017/000000226111.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000058636.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000458790.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000461275.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000268996.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000370999.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000481404.jpg', '/kaggle/input/coco-2017-dataset/coco2017/val2017/000000121153.jpg', '/kaggle

**FCN with 1DCNN**

In [ ]:
import os
import pandas as pd
import numpy as np
import torchaudio
import torch
import tensorflow as tf
import sklearn.metrics


train_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_features.npy")
train_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_labels.npy")
val_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_features.npy")
val_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_labels.npy")
test_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_features.npy")
test_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_labels.npy")
print(train_features.shape)
print(train_labels.shape)
print(val_features.shape)
print(val_labels.shape)
print(test_features.shape)
print(test_labels.shape)
train_features = np.expand_dims(train_features, axis=2)
val_features = np.expand_dims(val_features, axis=2)
test_features = np.expand_dims(test_features, axis=2)
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

In [ ]:
def create_fcn_model(input_shape, hidden_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hidden_size, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(hidden_size // 2, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(hidden_size // 4, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(hidden_size // 8, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])
    return model

# Define model hyperparameters
input_shape = (768,1) #WavLM
hidden_size = 1024
learning_rate = 0.0005
num_epochs = 20
batch_size = 32
model = create_fcn_model(input_shape, hidden_size)
print(model.summary())

In [ ]:
# Define learning rate scheduling callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the FCN model with learning rate scheduling
model.fit(train_features, train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          verbose=1,
          validation_data=(val_features, val_labels),
          callbacks=[lr_scheduler])

In [ ]:
predicted_scores = model.predict(test_features)  
# Create a new NumPy array with the condition
#pred_labels = tf.math.argmax(predicted_scores, axis=1)
print(predicted_scores.shape)


# Define the compute_eer function
def compute_eer(label, pred, positive_label=1):
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, pos_label=positive_label)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = (eer_1 + eer_2) / 2
    return eer

# Calculate EER using the compute_eer function
eer = compute_eer(test_labels, predicted_scores)

print("Equal Error Rate (EER):", eer)

**FCN Without 1D CNN**

In [ ]:
import os
import pandas as pd
import numpy as np
import torchaudio
import torch
import tensorflow as tf
import sklearn.metrics

train_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_features.npy")
train_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/train/wavlm_train_labels.npy")
val_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_features.npy")
val_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/val/wavlm_val_labels.npy")
test_features = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_features.npy")
test_labels = np.load("/kaggle/input/icasspembeddings/speech_icassp/speech/WavLM/Asvpoof_2019/test/wavlm_test_labels.npy")
print(train_features.shape)
print(train_labels.shape)
print(val_features.shape)
print(val_labels.shape)
print(test_features.shape)
print(test_labels.shape)
train_features = np.expand_dims(train_features, axis=2)
val_features = np.expand_dims(val_features, axis=2)
test_features = np.expand_dims(test_features, axis=2)
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

In [ ]:
# Define the FCN model architecture without 1DCNN
def create_fcn_model(input_shape, hidden_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),  # Flatten the input
        tf.keras.layers.Dense(hidden_size, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(hidden_size // 2, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(hidden_size // 4, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(hidden_size // 8, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define model hyperparameters
input_shape = (768,1) #WavLM
hidden_size = 1024
learning_rate = 0.0005
num_epochs = 20
batch_size = 32
model = create_fcn_model(input_shape, hidden_size)
print(model.summary())

In [ ]:
# Define learning rate scheduling callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Train the FCN model with learning rate scheduling
model.fit(train_features, train_labels,
          epochs=num_epochs,
          batch_size=batch_size,
          verbose=1,
          validation_data=(val_features, val_labels),
          callbacks=[lr_scheduler])

In [ ]:
predicted_scores = model.predict(test_features)  
# Create a new NumPy array with the condition
#pred_labels = tf.math.argmax(predicted_scores, axis=1)
print(predicted_scores.shape)


# Define the compute_eer function
def compute_eer(label, pred, positive_label=1):
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, pos_label=positive_label)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = (eer_1 + eer_2) / 2
    return eer

# Calculate EER using the compute_eer function
eer = compute_eer(test_labels, predicted_scores)

print("Equal Error Rate (EER):", eer)